# Code

In [ ]:
import torch
import torch.nn as nn
import torchvision 
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(0.2),
                                transforms.ToTensor(),
                                transforms.Resize((224,224))
                               ])

dataset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/5th sem/ML/Resized Data',
                                           transform = transform)
print("No of Classes: ", len(dataset.classes))

train, val = torch.utils.data.random_split(dataset, [1231, 300])

train_loader = torch.utils.data.DataLoader(dataset = train,
                                           batch_size = 32, 
                                           shuffle = True)

val_loader = torch.utils.data.DataLoader(dataset = val,
                                         batch_size = 32, 
                                         shuffle = True)

No of Classes:  4


In [ ]:
examples = enumerate(val_loader)
batch_idx, (example_data, example_targets) = next(examples)

import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i].numpy().transpose())
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])

In [ ]:
squeezenet = torchvision.models.squeezenet1_0(pretrained=True)

Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_0-b66bff10.pth


  0%|          | 0.00/4.78M [00:00<?, ?B/s]

In [ ]:
squeezenet

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [ ]:
squeezenet.classifier[3].out_features = 4
for param in squeezenet.features.parameters(): 
    param.requires_grad = False

squeezenet = squeezenet.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(squeezenet.parameters(), lr=0.001)

In [ ]:
total_step = len(train_loader)
Loss = []
Acc = []
Val_Loss = []
Val_Acc = []
n = 15

for epoch in range(n):
  acc = 0
  val_acc = 0
  for i, (images, labels) in enumerate(train_loader):
    squeezenet.train()
    images = images.to(device)
    labels = labels.to(device)
    
    # Forward pass
    outputs = squeezenet(images)
    loss = criterion(outputs, labels)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Checking accuracy
    preds = outputs.data.max(dim=1,keepdim=True)[1]
    acc += preds.eq(labels.data.view_as(preds)).cpu().sum()
    
  acc = acc/len(train_loader.dataset) * 100
    
  for i, (images, labels) in enumerate(val_loader):
    squeezenet.eval()
    images = images.to(device)
    labels = labels.to(device)
    
    # Forward pass
    outputs = squeezenet(images)
    val_loss = criterion(outputs, labels)
    
    # Checking accuracy
    preds = outputs.data.max(dim=1,keepdim=True)[1]
    val_acc += preds.eq(labels.data.view_as(preds)).cpu().sum()
    
  val_acc = val_acc/len(val_loader.dataset) * 100
    
  print("Epoch {} =>  loss : {loss:.2f};   Accuracy : {acc:.2f}%;   Val_loss : {val_loss:.2f};   Val_Accuracy : {val_acc:.2f}%".format(epoch+1, loss=loss.item(), acc=acc, val_loss=val_loss.item(), val_acc=val_acc))
  
  Loss.append(loss)
  Acc.append(acc)

  Val_Loss.append(val_loss)
  Val_Acc.append(val_acc)

Epoch 1 =>  loss : 1.12;   Accuracy : 31.11%;   Val_loss : 1.28;   Val_Accuracy : 57.00%
Epoch 2 =>  loss : 0.62;   Accuracy : 64.91%;   Val_loss : 0.52;   Val_Accuracy : 73.33%
Epoch 3 =>  loss : 0.61;   Accuracy : 76.52%;   Val_loss : 0.53;   Val_Accuracy : 82.67%
Epoch 4 =>  loss : 0.73;   Accuracy : 80.18%;   Val_loss : 0.26;   Val_Accuracy : 84.00%
Epoch 5 =>  loss : 0.69;   Accuracy : 84.65%;   Val_loss : 0.18;   Val_Accuracy : 84.00%
Epoch 6 =>  loss : 0.18;   Accuracy : 86.27%;   Val_loss : 0.11;   Val_Accuracy : 86.67%
Epoch 7 =>  loss : 0.22;   Accuracy : 87.98%;   Val_loss : 0.14;   Val_Accuracy : 88.67%
Epoch 8 =>  loss : 0.49;   Accuracy : 89.20%;   Val_loss : 0.26;   Val_Accuracy : 87.00%
Epoch 9 =>  loss : 0.60;   Accuracy : 90.74%;   Val_loss : 0.19;   Val_Accuracy : 90.00%
Epoch 10 =>  loss : 0.15;   Accuracy : 91.96%;   Val_loss : 0.18;   Val_Accuracy : 90.67%
Epoch 11 =>  loss : 0.08;   Accuracy : 90.98%;   Val_loss : 0.06;   Val_Accuracy : 89.33%
Epoch 12 =>  loss :

In [ ]:
plt.plot(range(n),Loss)
plt.plot(range(n),Val_Loss)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title("Loss")
plt.legend(["Training Loss", "Validation Loss"])
plt.show()

In [ ]:
plt.plot(range(n),Acc)
plt.plot(range(n),Val_Acc)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title("Accuracy")
plt.legend(["Training Accuracy", "Validation Accuracy"])
plt.show()

In [ ]:
Accuracies = []
squeezenet.eval()  
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:
        y_pred = []
        Images = images.to(device)
        Labels = labels.to(device)
        outputs = squeezenet(Images)
        prediction_array = outputs.data
        
        _, predicted = torch.max(prediction_array, 1)
        y_pred += predicted
        total += Labels.size(0)
        correct += (predicted == Labels).sum().item()

    acc = 100 * correct / total
    Accuracies.append(acc)
    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the test images: 91.33333333333333 %


# Save model


In [ ]:
from torch.utils.mobile_optimizer import optimize_for_mobile


example = torch.rand(1,3,224,224,device="cuda")

trace_script_module = torch.jit.trace(squeezenet,example)
trace_script_module_optimized = optimize_for_mobile(trace_script_module)
trace_script_module_optimized._save_for_lite_interpreter("/content/drive/MyDrive/5th sem/ML/Project/Models/model6.pt")